In [2]:
globals().clear()


from __future__ import (absolute_import, division, print_function, unicode_literals)
from builtins import * 
from io import BytesIO, TextIOWrapper
from zipfile import ZipFile
import urllib.request
import csv
from shapely.geometry import Point
import geopandas as gpd

import time
import numpy as np
import zarr
from shapely.geometry import Polygon, Point
import pandas as pd
import geopandas as gpd
import s3fs
import os


os.chdir('C:/Users/lbeatty/Documents/Lauren_MIP_Contribution/')


C:\Users\lbeatty\AppData\Local\Temp\ipykernel_25220\3896552863.py:11: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [3]:
def rect(i, w, s, e, n):
    x = [w[i], e[i], e[i], w[i], w[i]]
    y = [s[i], s[i], n[i], n[i], s[i]]
    return x, y

def poly(sr):
    ret = []
    w = sr["W"][:]
    s = sr["S"][:]
    e = sr["E"][:]
    n = sr["N"][:]
    for i in range(52411):
        x, y = rect(i, w, s, e, n)
        ret.append(Polygon([[x[0],y[0]],[x[1],y[1]],[x[2],y[2]],
                            [x[3],y[3]],[x[4],y[4]]]))
    return ret

In [5]:
rl = 's3://inmap-model/isrm_v1.2.1.zarr/'
url = 's3://inmap-model/isrm_v1.2.1.zarr/'

fs = s3fs.S3FileSystem(anon=True, client_kwargs=dict(region_name='us-east-2'))
sr = zarr.open(s3fs.S3Map(url, s3=fs, check=False), mode="r")

p = poly(sr)


Making polygons as geometry.


In [6]:
scenario = '26z-short-base-200'
model = 'GenX'

emis = gpd.read_file('InMap/MIP_Emissions/'+scenario+'/'+model+'/emissions_2020.shp')


In [7]:
df = pd.DataFrame({'Location': range(52411)})
gdf = gpd.GeoDataFrame(df, geometry=p)

# join the emis dataframe into the grid dataframe
emis.crs = "+proj=longlat"
gdf.crs = "+proj=lcc +lat_1=33.000000 +lat_2=45.000000 +lat_0=40.000000 +lon_0=-97.000000 +x_0=0 +y_0=0 +a=6370997.000000 +b=6370997.000000 +to_meter=1"
emis = emis.to_crs(gdf.crs)
join_right_df = gdf.sjoin(emis, how="right")
print("Finished joining the dataframes.")

Finished joining the dataframes.


In [8]:
print(join_right_df.head)

<bound method NDFrame.head of       index_left  Location   Longitude   Latitude          NOx          SOx  \
0          28522     28522    -88.0103    31.0069   628.021704    10.693294   
1          28522     28522    -88.0103    31.0069   628.021704    10.693294   
2          28522     28522    -88.0103    31.0069  1122.295464  1586.303117   
3          28522     28522    -88.0103    31.0069  2192.882493  3099.519194   
4          28522     28522    -88.0103    31.0069    56.572379     2.232006   
...          ...       ...         ...        ...          ...          ...   
7524       42990     42990    -76.6861    38.5444     0.007506     0.000144   
7525       25755     25755    -89.4635  31.837173     9.822716     0.188374   
7526       37495     37495  -80.694722  32.329402     1.320886     0.182576   
7527       37495     37495  -80.694722  32.329402     0.528355     0.073030   
7528       37495     37495  -80.694722  32.329402     1.268966     0.024335   

           PM2_5     